# Caching
### We look at caching in this notebook

In [ ]:
%run ../Includes/Classroom-Setup

Data mounted to /mnt/davis ...

In [ ]:
%fs ls /mnt/davis/fire-calls/fire-calls-truncated-comma.csv

path,name,size
dbfs:/mnt/davis/fire-calls/fire-calls-truncated-comma.csv,fire-calls-truncated-comma.csv,89222803


#### File size is 90MB in file system.

In [ ]:
%python 
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').appName('Cachingdf').getOrCreate()

In [ ]:
%python
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v2.4.3 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [ ]:
%python
df = spark.read.csv('/mnt/davis/fire-calls/fire-calls-truncated-comma.csv',header='true',inferSchema='true')

#### Count took 3sec when the dataframe is not cached

In [ ]:
%python
df.count()

Out[24]: 240613

#### Lets cache the dataframe 
#### Note cache is lazy in dataframe API so it will cache only after first action on the dataframe

In [ ]:
%python
df.cache()

Out[8]: DataFrame[Call Number: int, Unit ID: string, Incident Number: int, Call Type: string, Call Date: string, Watch Date: string, Received DtTm: string, Entry DtTm: string, Dispatch DtTm: string, Response DtTm: string, On Scene DtTm: string, Transport DtTm: string, Hospital DtTm: string, Call Final Disposition: string, Available DtTm: string, Address: string, City: string, Zipcode of Incident: int, Battalion: string, Station Area: string, Box: string, Original Priority: string, Priority: string, Final Priority: int, ALS Unit: boolean, Call Type Group: string, Number of Alarms: int, Unit Type: string, Unit sequence in call dispatch: int, Fire Prevention District: string, Supervisor District: string, Neighborhooods - Analysis Boundaries: string, Location: string, RowID: string]

#### count took 12 seconds because under the hood it's caching the entire dataframe in memory

In [ ]:
%python
df.count()

Out[9]: 240613

Now we can see entire dataframe is cached
## Spark UI

You'll notice that our data when cached actually takes up less space than our file on disk! That is thanks to the Tungsten Optimizer.

Our file in memory takes up ~59 MB, and on disk it takes up ~90 MB!

<div><img src="http://files.training.databricks.com/images/eLearning/ucdavis/inmemorysize.png" style="height: 300px; border: 1px solid #aaa; border-radius: 10px 10px 10px 10px; box-shadow: 5px 5px 5px #aaa; margin: 20px"/></div>

#### Lets count again
#### Now count took just 0.5sec because the entire dataframe is cached in memory

In [ ]:
%python
df.count()

Out[10]: 240613

#### We can also check the storage level
#### More about storage level [link](https://data-flair.training/blogs/pyspark-storagelevel/)

In [ ]:
%python
df.storageLevel

Out[12]: StorageLevel(True, True, False, True, 1)

#### Lets unpersist the dataframe

In [ ]:
%python
df.unpersist()

Out[16]: DataFrame[Call Number: int, Unit ID: string, Incident Number: int, Call Type: string, Call Date: string, Watch Date: string, Received DtTm: string, Entry DtTm: string, Dispatch DtTm: string, Response DtTm: string, On Scene DtTm: string, Transport DtTm: string, Hospital DtTm: string, Call Final Disposition: string, Available DtTm: string, Address: string, City: string, Zipcode of Incident: int, Battalion: string, Station Area: string, Box: string, Original Priority: string, Priority: string, Final Priority: int, ALS Unit: boolean, Call Type Group: string, Number of Alarms: int, Unit Type: string, Unit sequence in call dispatch: int, Fire Prevention District: string, Supervisor District: string, Neighborhooods - Analysis Boundaries: string, Location: string, RowID: string]

#### Now check the storage level after unpersist

In [ ]:
%python
df.storageLevel

Out[17]: StorageLevel(False, False, False, False, 1)

#### Count took 3sec again since dataframe is not cached

In [ ]:
%python
df.count()

Out[18]: 240613